## Глава 1. 
### 1.1 Вступление
В этой работе я не буду пытаться предсказать цену. На мой взгляд это невозможно или очень трудно. Я сосредоточусь на том, чтоб попытаться по тем данным, которые предоставляет биржа, определитять силу движения, силу свечи.  Прежде всего по объему торгов. Анализировать момнету Dogicoin на  timeframe 15 минут.

Для начала подготовим данные к машинному обучению.

In [90]:
import pandas as pd
df = pd.read_csv('DOGEUSDT_2020-11-19_2024-11-19_INTERVAL_15MINUTE.csv')
df.head()

,Open time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,2020-11-19 19:00:00,0.002931,0.002934,0.002896,0.002904,19760566.0,1605813299999,57614.417807,505,4176342.0,12142.156017,0
1,2020-11-19 19:15:00,0.002909,0.002913,0.002890,0.002898,7715960.0,1605814199999,22327.291478,114,5646231.0,16335.192524,0
2,2020-11-19 19:30:00,0.002900,0.002911,0.002897,0.002911,6004543.0,1605815099999,17439.155566,67,2030787.0,5898.155289,0
3,2020-11-19 19:45:00,0.002910,0.002917,0.002907,0.002916,1476961.0,1605815999999,4300.449462,28,1021437.0,2975.096665,0
4,2020-11-19 20:00:00,0.002910,0.002910,0.002900,0.002902,7644641.0,1605816899999,22187.315007,81,4592669.0,13323.285198,0


### 1.2 Подготвка данных

1. Volume:

    Это общее количество DOGE, которое было куплено и продано за данный временной интервал.
    - Измеряется в базовом активе (DOGE).

2. Quote Asset Volume:

    Это объем в котируемой валюте (USDT), эквивалентный проторгованному объему базового актива.\
    Вычисляется как:
    - Quote Asset Volume = Volume × Средняя цена сделки за интервал

3. Taker Buy Base Asset Volume:

    - Это объем базового актива, который был куплен с использованием ордеров типа "Taker" (рыночных ордеров).

4. Taker Buy Quote Asset Volume:
    - Это объем котируемой валюты, потраченной на покупку базового актива ордерами "Taker". 

In [91]:
df.drop(['Close Time', 'Ignore'], axis=1, inplace=True)

In [92]:
df[df.columns[5:]].corr()

,Volume,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume
Volume,1.000000,0.594776,0.630057,0.996246,0.593225
Quote Asset Volume,0.594776,1.000000,0.951328,0.592700,0.995905
Number of Trades,0.630057,0.951328,1.000000,0.628020,0.948987
Taker Buy Base Asset Volume,0.996246,0.592700,0.628020,1.000000,0.596114
Taker Buy Quote Asset Volume,0.593225,0.995905,0.948987,0.596114,1.000000


Удалим из датасета значения:
- Taker Buy Quote Asset Volume (сильная кор с Quote Asset Volume). И зачем рассматривать ордера типа Мейкер когда надо онализировать все ордера
- Taker Buy Base Asset Volume


In [93]:
df.drop(['Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume'], axis=1, inplace=True)

In [96]:
# Теперь построим относительные изменения и построим 2 модли
# Процентное изменене с момента open-close
# Процентное изменене с момента High-Low
# Абсолютное процентное изменение High-Low

# df['open_close_%'] = round((abs(df['Close'] - df['Open']) / df['Open']) * 100, 2)

df['hifg_low_%'] = round((abs(df['High'] - df['Low']) / df['Open']) * 100, 2)


df.head()


,Open time,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,open_close_%,hifg_low_%
0,2020-11-19 19:00:00,0.002931,0.002934,0.002896,0.002904,19760566.0,57614.417807,505,0.93,1.29
1,2020-11-19 19:15:00,0.002909,0.002913,0.002890,0.002898,7715960.0,22327.291478,114,0.36,0.78
2,2020-11-19 19:30:00,0.002900,0.002911,0.002897,0.002911,6004543.0,17439.155566,67,0.39,0.51
3,2020-11-19 19:45:00,0.002910,0.002917,0.002907,0.002916,1476961.0,4300.449462,28,0.20,0.33
4,2020-11-19 20:00:00,0.002910,0.002910,0.002900,0.002902,7644641.0,22187.315007,81,0.27,0.35


In [97]:
df['Future_open_close'] = df['hifg_low_%'].shift(-1)
df = df.dropna()
df.head()

,Open time,Open,High,Low,Close,Volume,Quote Asset Volume,Number of Trades,open_close_%,hifg_low_%,Future_open_close
0,2020-11-19 19:00:00,0.002931,0.002934,0.002896,0.002904,19760566.0,57614.417807,505,0.93,1.29,0.78
1,2020-11-19 19:15:00,0.002909,0.002913,0.002890,0.002898,7715960.0,22327.291478,114,0.36,0.78,0.51
2,2020-11-19 19:30:00,0.002900,0.002911,0.002897,0.002911,6004543.0,17439.155566,67,0.39,0.51,0.33
3,2020-11-19 19:45:00,0.002910,0.002917,0.002907,0.002916,1476961.0,4300.449462,28,0.20,0.33,0.35
4,2020-11-19 20:00:00,0.002910,0.002910,0.002900,0.002902,7644641.0,22187.315007,81,0.27,0.35,0.51


In [43]:
# разделим данные
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, shuffle=False)

In [98]:
X_train, y_train = train.drop(['hifg_low_%', 'open_close_%', 'Open time', 'Future_open_close'], axis=1).to_numpy(), train['Future_open_close'].to_numpy()
X_test, y_test = test.drop(['hifg_low_%', 'open_close_%', 'Open time', 'Future_open_close'], axis=1).to_numpy(), test['Future_open_close'].to_numpy()

Построим 2 модели. Одна будет предсказывать умеренный результат(% изменения на закрытие), другая будет предсказывать процент диапазон. Насколько сильно цена может отклониться. 

## Моделирование с помощью CatBoost

### Метод 1

Сделаем упрощение и предположим, что цена на криптовалюты зависит только от объема в текущий момент и не зависит от предыдущих значений. Тогда, для прогназирования изменения цены воспользуемся методом градиентного бустинга

In [99]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [100]:
# Создание модели
model = CatBoostRegressor(
    iterations=100,          # Количество итераций
    learning_rate=0.1,       # Скорость обучения
    depth=7,                 # Глубина дерева
    verbose=50               # Лог каждые 50 итераций
)

# Обучение модели
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")

0:	learn: 0.7513668	total: 7.35ms	remaining: 728ms
50:	learn: 0.6104752	total: 313ms	remaining: 301ms
99:	learn: 0.5996106	total: 614ms	remaining: 0us
Mean Squared Error: 0.15
Mean Absolute Error: 0.24


In [26]:
y_train.var()

0.6009650800825099

In [48]:
from sklearn.model_selection import GridSearchCV
# Определяем модель без параметров
model = CatBoostRegressor(verbose=0)

# Определяем сетку гиперпараметров
param_grid = {
    'iterations': [100, 200],          # Количество итераций
    'learning_rate': [0.01, 0.05,],    # Скорость обучения
    'depth': [4, 6, 8],                    # Глубина деревьев
    'l2_leaf_reg': [1, 3, 5],              # Регуляризация L2
    'bagging_temperature': [0.0, 0.5], # Температура семплирования (бустинг)
}

# GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Оценка по MSE
    cv=3,                              # Кросс-валидация на 3 фолдах
    verbose=1,
    n_jobs=-1                          # Используем все доступные ядра
)

# Запускаем поиск
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Лучшая модель
best_model = grid_search.best_estimator_

# Оценка на тестовых данных
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (лучшей модели): {mse:.2f}")

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Лучшие параметры: {'bagging_temperature': 0.0, 'depth': 6, 'iterations': 100, 'l2_leaf_reg': 1, 'learning_rate': 0.05}
Mean Squared Error (лучшей модели): 0.14


In [ ]:
# bagging_temperature — Температура семплирования:

# Контролирует случайность семплирования данных на каждой итерации.

In [130]:
iter = -1004

df.iloc[iter]

Open time             2024-11-09 07:45:00
Open                              0.20568
High                                 0.21
Low                               0.20564
Close                             0.20925
Volume                        129148613.0
Quote Asset Volume         26940413.67988
Number of Trades                    38907
open_close_%                         1.74
hifg_low_%                           2.12
Future_open_close                    1.01
Name: 139157, dtype: object

In [131]:
price = df.iloc[iter]['Open']
precent = model.predict(df.iloc[-6].drop(['hifg_low_%', 'open_close_%', 'Open time', 'Future_open_close']))  / 100
high = (1 + precent) * price
low = (1 - precent) * price
print(f'''
Нижняя гроница цены : {low:.5f}
Верхняя границы : {high:.5f}''')


Нижняя гроница цены : 0.20380
Верхняя границы : 0.20756


In [103]:
(precent/100 + 1) * price

0.39937653875637746

In [66]:
# 1.7 %
# Данные
price = 0.4141  # Текущая цена актива
percent_growth = 1.7  # Процент роста

# Вычисление абсолютного роста
absolute_increase = price * (percent_growth / 100)

# Результат
print(f"Актив вырос на {absolute_increase:.4f}")


Актив вырос на 0.0070


In [67]:
absolute_increase +  price

0.4211397

## VAR
### Векторная авторегрессия (VAR) 

**VAR (Vector AutoRegression)** — это модель для анализа временных рядов, которая учитывает взаимосвязь между несколькими переменными. Она представляет собой обобщение авторегрессии (AR), но для многомерных временных рядов, то есть включает сразу несколько переменных, каждая из которых может быть предсказана на основе её собственной истории и историй других переменных.

#### Принципы работы VAR:
- Каждая переменная в модели представлена как линейная функция лагов всех переменных во временном ряду, включая её саму.
- Подразумевается, что все переменные имеют взаимное влияние (endogeneity), то есть модель не требует разделения переменных на объясняемые и объясняющие.

#### Условия применения модели:
1. **Стационарность временных рядов:**
   - Все временные ряды, используемые в модели, должны быть стационарными, что означает, что их среднее, дисперсия и ковариация остаются постоянными во времени.
   - Для проверки стационарности используется **тест Дики-Фуллера** (ADF-тест).
   - Если ряды не стационарны, применяется преобразование (например, взятие первых разностей).

2. **Отсутствие мультиколлинеарности:**
   - Переменные не должны быть слишком сильно скоррелированы на всех уровнях. Это необходимо для точного вычисления коэффициентов модели.

3. **Выбор лагов:**
   - Количество лагов определяется с помощью информационных критериев (например, AIC, BIC, HQIC).
   - Выбор оптимального количества лагов критически важен для корректного построения модели.

#### Структура VAR:
Допустим, у вас есть две переменные \(y_t\) и \(x_t\). Тогда уравнения VAR выглядят следующим образом:

\[
y_t = c_1 + a_{11} y_{t-1} + a_{12} x_{t-1} + \epsilon_{1t}
\]
\[
x_t = c_2 + a_{21} y_{t-1} + a_{22} x_{t-1} + \epsilon_{2t}
\]

Где:
- \(c_1, c_2\) — константы.
- \(a_{ij}\) — коэффициенты авторегрессии.
- \(\epsilon_{1t}, \epsilon_{2t}\) — ошибки модели.

#### Как подготовить данные:
1. Собрать временные ряды для всех переменных, которые будут включены в модель.
2. Провести тесты на стационарность и преобразовать данные, если это необходимо.
3. Проанализировать кросс-корреляцию между переменными, чтобы оценить взаимосвязи.
4. Нормализовать данные (если требуется), чтобы избежать искажений из-за масштаба.

#### Преимущества VAR:
- Учитывает взаимное влияние всех переменных.
- Не требует строгих гипотез о причинно-следственной связи между переменными.
- Хорошо подходит для моделирования сложных систем.

#### Ограничения VAR:
- Требует больших объемов данных для получения надежных оценок.
- Чувствительна к выбору количества лагов.
- Модели VAR плохо интерпретируемы, так как все переменные рассматриваются как эндогенные.

#### Пример областей применения:
- Экономика: анализ взаимосвязей между макроэкономическими показателями (например, ВВП, инфляция, процентные ставки).
- Финансы: оценка влияния изменений цен на активы.
- Социальные науки: исследование влияния одного индикатора (например, безработицы) на другой (например, уровень преступности).

Если вас интересуют детали реализации, настройка модели или интерпретация результатов, дайте знать, я могу привести пример или описать алгоритм анализа!

### VARIMA (Vector ARIMA)

**VARIMA (Vector Autoregressive Integrated Moving Average)** — это обобщение модели ARIMA для многомерных временных рядов. VARIMA сочетает возможности VAR (векторной авторегрессии) и ARIMA (авторегрессии, интегрированного скользящего среднего), что делает её мощным инструментом для работы с многомерными временными рядами, которые могут быть нестационарными.

---

#### Структура VARIMA:
Модель VARIMA используется, когда многомерные временные ряды имеют взаимосвязь между переменными, но при этом они не стационарны и требуют интеграции (то есть дифференцирования) для достижения стационарности. 

VARIMA объединяет три компонента:
1. **AR (авторегрессия):** использование лагов собственных и других переменных для предсказания текущего значения.
2. **I (интеграция):** устранение трендов или сезонности для достижения стационарности путём дифференцирования временного ряда.
3. **MA (скользящее среднее):** моделирование случайных ошибок как линейной комбинации прошлых ошибок.

---

#### Уравнение VARIMA:
Для переменных \( y_t \) и \( x_t \), VARIMA может быть записана следующим образом:

1. Преобразование данных для стационарности через дифференцирование:
   \[
   y_t' = y_t - y_{t-1}, \quad x_t' = x_t - x_{t-1}
   \]

2. После достижения стационарности применяется модель VARMA:
   \[
   y_t' = c_1 + \sum_{i=1}^{p} a_{1i} y_{t-i}' + \sum_{i=1}^{p} b_{1i} x_{t-i}' + \sum_{j=1}^{q} \theta_{1j} \epsilon_{t-j} + \epsilon_{1t}
   \]
   \[
   x_t' = c_2 + \sum_{i=1}^{p} a_{2i} y_{t-i}' + \sum_{i=1}^{p} b_{2i} x_{t-i}' + \sum_{j=1}^{q} \theta_{2j} \epsilon_{t-j} + \epsilon_{2t}
   \]

Где:
- \( y_t, x_t \) — анализируемые временные ряды.
- \( p \) — порядок авторегрессии.
- \( q \) — порядок скользящего среднего.
- \( \epsilon_t \) — ошибки модели.
- \( \theta \) — параметры скользящего среднего.

---

#### Условия применения VARIMA:
1. **Необходимость интеграции (I):**
   - Если ряды не стационарны, но становятся стационарными после одного или нескольких дифференцирований, можно использовать VARIMA. 
   - Это проверяется с помощью теста Дики-Фуллера (ADF) или KPSS.

2. **Взаимосвязь переменных:**
   - Все временные ряды должны быть связаны друг с другом и влиять друг на друга.

3. **Выбор параметров (p, d, q):**
   - \( p \) — количество лагов в авторегрессии. Определяется с помощью информационных критериев (например, AIC или BIC).
   - \( d \) — порядок дифференцирования, чтобы сделать ряды стационарными.
   - \( q \) — количество лагов в скользящем среднем. 

4. **Объём данных:**
   - Модель VARIMA требует большого объёма данных для точной оценки параметров, особенно при высокой размерности временного ряда.

---

#### Процесс работы с VARIMA:
1. **Подготовка данных:**
   - Проверьте временные ряды на стационарность. Если ряды нестационарны, выполните дифференцирование.
   - Проверьте взаимосвязь между рядами (например, с помощью анализа корреляции или теста на причинность Грейнджера).

2. **Определение параметров:**
   - Используйте графики автокорреляции (ACF) и частичной автокорреляции (PACF) для выбора параметров \( p \) и \( q \).
   - Определите \( d \), исходя из необходимости интеграции.

3. **Построение модели:**
   - Настройте модель VARIMA с выбранными параметрами.
   - Оцените параметры с помощью метода наименьших квадратов или другого подходящего алгоритма.

4. **Оценка качества:**
   - Проверьте адекватность модели с помощью тестов на автокорреляцию остатков, информационных критериев и прогностической точности.

---

#### Преимущества VARIMA:
- Позволяет моделировать сложные взаимосвязи между несколькими временными рядами.
- Учитывает нестационарность данных.
- Более точная, чем VAR, при наличии трендов и сезонности.

#### Ограничения VARIMA:
- Требует больших объёмов данных.
- Сложность выбора параметров \( p, d, q \).
- Не подходит для данных с высокой сезонностью, если её нельзя эффективно устранить.

---

#### Пример применения:
1. **Экономика:** анализ взаимосвязи инфляции, ВВП и уровня безработицы, с учётом долгосрочных трендов.
2. **Финансы:** прогнозирование динамики нескольких финансовых показателей, таких как акции, облигации и валюты.
3. **Энергетика:** моделирование взаимосвязи потребления энергии, цен на нефть и углеродных выбросов.

Если нужны примеры кода или практическое применение модели, напишите!

Temporal Fusion Transformer (TFT):
TFT специально создан для задач временных рядов. Его ключевые особенности:

Интеграция разнородных данных:
Обрабатывает временные и статические признаки, что особенно полезно в бизнес-аналитике, экономике и финансах.
Многоголовочный механизм внимания (Multi-Head Attention):
Определяет, какие временные моменты и переменные важны для прогноза.
Селективный механизм пропуска информации (Gating Mechanism):
Позволяет TFT сосредотачиваться на важных данных, минимизируя влияние незначительных сигналов.
Обработка неопределённости:
TFT предоставляет вероятностные прогнозы, оценивая диапазон возможных значений.
Применение трансформеров к временным рядам:
Обработка входных данных:

Временной ряд разбивается на окна фиксированной длины (например, по неделям, месяцам).
Признаки разделяются на:
Статические: остаются постоянными (например, ID объекта, географическая зона).
Динамические: меняются с течением времени (например, температура, спрос).
Подготовка данных:

Нормализация или стандартизация временных рядов.
Кодирование категориальных признаков (например, one-hot encoding или embedding).
Тренировка модели:

Трансформеры обучаются на временных окнах, где часть данных используется как вход (история), а другая — для валидации прогнозов (будущее).
Прогнозирование:

Модель предсказывает значения для следующих временных шагов на основе исторических данных.


Линейная регрессия применяется для моделирования линейной зависимости между зависимой переменной (откликом) и одной или несколькими независимыми переменными (предикторами). Для корректного использования линейной регрессии ряд данных должен удовлетворять следующим условиям:

1. **Линейность связи**: Между зависимой и независимой переменными должна существовать линейная зависимость. Это означает, что изменение независимой переменной приводит к пропорциональному изменению зависимой.

2. **Независимость наблюдений**: Все наблюдения должны быть статистически независимыми друг от друга. Это важно для обеспечения корректности статистических выводов.

3. **Гомоскедастичность**: Дисперсия ошибок (остатков) должна быть постоянной для всех значений независимой переменной. То есть величина разброса остатков не должна зависеть от значения предиктора.

4. **Нормальность остатков**: Остатки (разницы между наблюдаемыми и прогнозируемыми значениями) должны быть распределены нормально с нулевым средним. Это условие важно для проведения статистических тестов и построения доверительных интервалов.

5. **Отсутствие мультиколлинеарности** (для множественной регрессии): Независимые переменные не должны быть сильно коррелированы между собой. Высокая корреляция между предикторами может привести к нестабильности оценок коэффициентов.

6. **Отсутствие автокорреляции**: Ошибки не должны быть коррелированы между собой, особенно в временных рядах. Наличие автокорреляции может искажать стандартные ошибки коэффициентов.

7. **Измерение без ошибок**: Предполагается, что независимые переменные измерены точно и без ошибок. Ошибки в измерении предикторов могут привести к смещению оценок.

8. **Отсутствие выбросов и влиятельных точек**: Аномальные наблюдения могут существенно повлиять на результаты регрессии. Необходимо выявлять и анализировать такие точки.

Перед применением линейной регрессии рекомендуется провести предварительный анализ данных:

- **Визуализация данных**: Построение графиков (например, диаграмм рассеяния) для проверки линейности и выявления выбросов.
- **Анализ остатков**: Проверка гомоскедастичности и нормальности распределения остатков.
- **Тесты на мультиколлинеарность**: Вычисление факторов инфляции дисперсии (VIF) для обнаружения корреляции между предикторами.

Если какие-либо из этих условий не выполняются, могут потребоваться преобразования данных, использование других моделей (например, нелинейной регрессии) или применение методов, устойчивых к нарушениям предположений.